In [1]:
import time
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
Data = joblib.load('dfcp.sav')

In [3]:
df = Data.copy()

In [4]:
df

,Title,Body,Tags,Body_words,Title_words,Body_words_join,Title_words_join,SL_Body,SL_Title
0,Qt FullScreen on Startup,I want to start an application on fullscreen (...,<c++>,"[start, application, fullscreen, MacOS, 10, 8,...","[Qt, FullScreen, Startup]",start application fullscreen MacOS 10 8 x Qt d...,Qt FullScreen Startup,"[start, applic, fullscreen, maco, 10, 8, x, qt...","[qt, fullscreen, startup]"
1,jQuery or Javascript - how to disable window s...,Hi is it possible to disable window scrolling ...,<javascript><jquery><css>,"[Hi, possible, disable, window, scrolling, wit...","[Javascript, disable, window, scroll, overflow...",Hi possible disable window scrolling without o...,Javascript disable window scroll overflow hidden,"[hi, possibl, disabl, window, scroll, without,...","[javascript, disabl, window, scroll, overflow,..."
2,Python mock: mocking base class for inheritance,I am testing a class that inherits from anothe...,<python>,"[testing, inherits, another, complex, DB, conn...","[mock, mocking, base, inheritance]",testing inherits another complex DB connection...,mock mocking base inheritance,"[test, inherit, anoth, complex, db, connect, m...","[mock, mock, base, inherit]"
3,Calling CreateProcessAsUser from C#,I've been attempting to create a new process u...,<c#>,"[attempting, create, process, context, specifi...","[Calling, CreateProcessAsUser]",attempting create process context specific Cre...,Calling CreateProcessAsUser,"[attempt, creat, process, context, specif, cre...","[call, createprocessasus]"
4,How does Facebook keep the header and footer f...,When browsing through Facebook pages the heade...,<javascript><html>,"[When, browsing, Facebook, pages, header, fixe...","[Facebook, keep, header, footer, fixed, loadin...",When browsing Facebook pages header fixed foot...,Facebook keep header footer fixed loading diff...,"[when, brow, facebook, page, header, fix, foot...","[facebook, keep, header, footer, fix, load, di..."
...,...,...,...,...,...,...,...,...,...
49994,Why would you choose the Java programming lang...,Why would you choose java over others? Why did...,<java>>,"[Why, choose, others, Why, choose, program, ap...","[would, choose, programming, language, others]",Why choose others Why choose program applicati...,would choose programming language others,"[whi, choos, other, whi, choos, program, appli...","[would, choos, program, languag, other]"
49995,How do I do a deep copy of an element in LINQ ...,I want to make a deep copy of a LINQ to XML XE...,<.net>,"[make, deep, copy, LINQ, XML, XElement, reason...","[deep, copy, element, LINQ, XML]",make deep copy LINQ XML XElement reason nodes ...,deep copy element LINQ XML,"[make, deep, copi, linq, xml, xelement, reason...","[deep, copi, element, linq, xml]"
49996,Steps to make a LED blink from a C/C++ program?,What are the easiest steps to make a small cir...,<c++><c>,"[easiest, steps, make, small, circuit, LED, fl...","[Steps, LED, blink, program]",easiest steps make small circuit LED flash pro...,Steps LED blink program,"[easiest, step, make, small, circuit, led, fla...","[step, led, blink, program]"
49997,Passing List<> to SQL Stored Procedure,I've often had to load multiple items to a par...,<c#><sql><sql-server>,"[often, load, multiple, items, particular, rec...","[Passing, List, Stored, Procedure]",often load multiple items particular record da...,Passing List Stored Procedure,"[often, load, multipl, item, particular, recor...","[pas, list, store, procedur]"


In [5]:
df['question'] = df['SL_Body'] + df['SL_Title']

In [6]:
df['Tags'] = df['Tags'].str.strip('<>').str.split('><')

In [7]:
df['Tags']

0                            [c++]
1        [javascript, jquery, css]
2                         [python]
3                             [c#]
4               [javascript, html]
                   ...            
49994                       [java]
49995                       [.net]
49996                     [c++, c]
49997        [c#, sql, sql-server]
49998                 [javascript]
Name: Tags, Length: 35346, dtype: object

In [8]:
mlb = MultiLabelBinarizer()

# Apply MultiLabelBinarizer to 'Tags' column
tags_df = pd.DataFrame(mlb.fit_transform(df['Tags']), columns=mlb.classes_, index=df.index)

# Concatenate original DataFrame with tags DataFrame
df = pd.concat([df, tags_df], axis=1)

In [9]:
df['question'] = df['question'].apply(' '.join)

In [10]:
X = df.question
Y = tags_df

# Unsupervided Method

## LDA

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df = 0.01 , stop_words = 'english')
X_fited = tfidf_vec.fit_transform(X)

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 10,max_iter=5,learning_method='online',learning_offset=50,random_state = 0).fit(X_fited)

In [12]:
 def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tfidf_vec.get_feature_names_out(), no_top_words)

Topic 0:
project applic net librari build web develop test run version
Topic 1:
std object compil method const variabl implement static differ void
Topic 2:
tabl column queri sql databas select row mysql insert store
Topic 3:
thread process line run print window program output command read
Topic 4:
self method object request foo void string properti privat test
Topic 5:
array list number string element valu item key integ iter
Topic 6:
imag div git color cs width height style text background
Topic 7:
instal directori file modul path py python packag lib import
Topic 8:
page html button event form javascript click jqueri url input
Topic 9:
date datetim convert format json day time 12 serial 11


## NMF

In [13]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, random_state=1, l1_ratio=.5, init='nndsvd')
nmf.fit(X_fited)

NMF(init='nndsvd', l1_ratio=0.5, n_components=10, random_state=1)

In [14]:
no_top_words = 1
display_topics(nmf, tfidf_vec.get_feature_names_out(), no_top_words)

Topic 0:
page
Topic 1:
div
Topic 2:
array
Topic 3:
tabl
Topic 4:
imag
Topic 5:
run
Topic 6:
git
Topic 7:
object
Topic 8:
list
Topic 9:
test


# Supervised Method

## Train/Test Split

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list(X), Y, test_size=0.33, random_state=42)

## TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Scores

In [17]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import jaccard_score

# list of unique tags
categories = list(Y.columns.values)

# list of metrics used to compare model performance
scores = ['f1','accuracy','precision','recall','fitting_duration', 'jaccard']

DfScore = pd.DataFrame(columns=['NameModel','f1','accuracy','precision','recall','fitting_duration', 'jaccard'])


# scores dataframe for MultioutputClassifier
LinearSVC_scores_moc = pd.DataFrame(columns=scores)
MultinomialNB_scores_moc = pd.DataFrame(columns=scores)
Logistic_regression_scores_moc = pd.DataFrame(columns=scores)

# scores dataframe for chain classifier
LinearSVC_scores_ch = pd.DataFrame(columns=scores)
MultinomialNB_scores_ch = pd.DataFrame(columns=scores)
Logistic_regression_scores_ch = pd.DataFrame(columns=scores)

# Classifier Chain

## LinearSVC

In [18]:
from sklearn.svm import LinearSVC
from sklearn.multioutput import ClassifierChain
base_clf = LinearSVC(random_state=0,max_iter=1000, tol=1e-5)
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [19]:
prediction = chain.predict(X_test)

In [20]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

LinearSVC_scores_ch['f1']=f1
LinearSVC_scores_ch['accuracy']=accuracy
LinearSVC_scores_ch['precision']=precision
LinearSVC_scores_ch['recall'] = recall
LinearSVC_scores_ch['fitting_duration'] = end-start
LinearSVC_scores_ch['jaccard'] = jaccard 

In [21]:
LinearSVC_scores_ch

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.651025,0.494128,0.723946,0.611757,2.471165,0.502229


In [22]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'LinearSVC1', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]


## MultinomialNB

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import ClassifierChain
base_clf = MultinomialNB()
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [24]:
prediction = chain.predict(X_test)

In [25]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

MultinomialNB_scores_ch['f1']=f1
MultinomialNB_scores_ch['accuracy']=accuracy
MultinomialNB_scores_ch['precision']=precision
MultinomialNB_scores_ch['recall'] = recall
MultinomialNB_scores_ch['fitting_duration'] = end-start
MultinomialNB_scores_ch['jaccard'] = jaccard 

C:\Users\aikim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
MultinomialNB_scores_ch

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.013696,0.008573,0.364002,0.007094,0.650097,0.007091


In [27]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'MultinomialNB1', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]
1,MultinomialNB1,[0.013696334111196468],[0.008572653236176597],[0.36400217929218404],[0.007093596059113301],0.650097,[0.007090934068848554]


## Logistic regression

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
base_clf = LogisticRegression(max_iter=500,tol=1e-5)
chain = ClassifierChain(base_clf, order='random', random_state=0)
start = time.time()
chain.fit(X_train,y_train)
end = time.time()

In [29]:
prediction = chain.predict(X_test)

In [30]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

Logistic_regression_scores_ch['f1']=f1
Logistic_regression_scores_ch['accuracy']=accuracy
Logistic_regression_scores_ch['precision']=precision
Logistic_regression_scores_ch['recall'] = recall
Logistic_regression_scores_ch['fitting_duration'] = end-start
Logistic_regression_scores_ch['jaccard'] = jaccard 

In [31]:
Logistic_regression_scores_ch

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.566711,0.40763,0.783984,0.478161,29.733469,0.417635


In [32]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'Logistic regression1', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]
1,MultinomialNB1,[0.013696334111196468],[0.008572653236176597],[0.36400217929218404],[0.007093596059113301],0.650097,[0.007090934068848554]
2,Logistic regression1,[0.5667107622025461],[0.4076296613801972],[0.7839842731141168],[0.4781609195402299],29.733469,[0.4176351894464903]


# MultiOutputClassifier

## LinearSVC

In [33]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
clf = MultiOutputClassifier(LinearSVC(random_state=0, tol=1e-5),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [34]:
prediction = clf.predict(X_test)

In [35]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

LinearSVC_scores_moc['f1']=f1
LinearSVC_scores_moc['accuracy']=accuracy
LinearSVC_scores_moc['precision']=precision
LinearSVC_scores_moc['recall'] = recall
LinearSVC_scores_moc['fitting_duration'] = end-start
LinearSVC_scores_moc['jaccard'] = jaccard

In [36]:
LinearSVC_scores_moc

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.654031,0.457608,0.79106,0.566634,4.56851,0.50693


In [37]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'LinearSVC2', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]
1,MultinomialNB1,[0.013696334111196468],[0.008572653236176597],[0.36400217929218404],[0.007093596059113301],0.650097,[0.007090934068848554]
2,Logistic regression1,[0.5667107622025461],[0.4076296613801972],[0.7839842731141168],[0.4781609195402299],29.733469,[0.4176351894464903]
3,LinearSVC2,[0.6540309437198691],[0.45760822974710674],[0.7910597202561167],[0.5666338259441708],4.568510,[0.5069299169725915]


## MultinomialNB

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
clf = MultiOutputClassifier(MultinomialNB(),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [39]:
prediction = clf.predict(X_test)

In [40]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

MultinomialNB_scores_moc['f1']=f1
MultinomialNB_scores_moc['accuracy']=accuracy
MultinomialNB_scores_moc['precision']=precision
MultinomialNB_scores_moc['recall'] = recall
MultinomialNB_scores_moc['fitting_duration'] = end-start
MultinomialNB_scores_moc['jaccard'] = jaccard

C:\Users\aikim\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
MultinomialNB_scores_moc

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.011855,0.007544,0.28796,0.006108,0.323445,0.006106


In [42]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'MultinomialNB2', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]
1,MultinomialNB1,[0.013696334111196468],[0.008572653236176597],[0.36400217929218404],[0.007093596059113301],0.650097,[0.007090934068848554]
2,Logistic regression1,[0.5667107622025461],[0.4076296613801972],[0.7839842731141168],[0.4781609195402299],29.733469,[0.4176351894464903]
3,LinearSVC2,[0.6540309437198691],[0.45760822974710674],[0.7910597202561167],[0.5666338259441708],4.568510,[0.5069299169725915]
4,MultinomialNB2,[0.011855141944311738],[0.007543934847835405],[0.28796015325670493],[0.006108374384236453],0.323445,[0.006105895149498925]


## Logistic regression

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
clf = MultiOutputClassifier(LogisticRegression(max_iter=500,tol=1e-5),n_jobs=-1)
start = time.time()
clf.fit(X_train,y_train)
end = time.time()

In [44]:
prediction = clf.predict(X_test)

In [45]:
f1 = []
accuracy = []
precision = []
recall = []
jaccard = []

f1.append(f1_score(y_test,prediction,average='weighted'))
accuracy.append(accuracy_score(y_test,prediction))
precision.append(precision_score(y_test,prediction,average='weighted'))
recall.append(recall_score(y_test,prediction,average='weighted'))
jaccard.append(jaccard_score(y_test, prediction, average='weighted'))  # Compute Jaccard score

Logistic_regression_scores_moc['f1']=f1
Logistic_regression_scores_moc['accuracy']=accuracy
Logistic_regression_scores_moc['precision']=precision
Logistic_regression_scores_moc['recall'] = recall
Logistic_regression_scores_moc['fitting_duration'] = end-start
Logistic_regression_scores_moc['jaccard'] = jaccard

In [46]:
Logistic_regression_scores_moc

,f1,accuracy,precision,recall,fitting_duration,jaccard
0,0.524274,0.339649,0.833773,0.399409,7.205719,0.375864


In [47]:
DfScore.loc[len(DfScore.index)] = {'NameModel':'Logistic regression2', 'f1':f1, 'accuracy':accuracy, 'precision':precision, 'recall':recall, 'fitting_duration':end-start, 'jaccard':jaccard}
DfScore

,NameModel,f1,accuracy,precision,recall,fitting_duration,jaccard
0,LinearSVC1,[0.6510249754411733],[0.494127732533219],[0.7239464978569428],[0.6117569786535304],2.471165,[0.5022289949873799]
1,MultinomialNB1,[0.013696334111196468],[0.008572653236176597],[0.36400217929218404],[0.007093596059113301],0.650097,[0.007090934068848554]
2,Logistic regression1,[0.5667107622025461],[0.4076296613801972],[0.7839842731141168],[0.4781609195402299],29.733469,[0.4176351894464903]
3,LinearSVC2,[0.6540309437198691],[0.45760822974710674],[0.7910597202561167],[0.5666338259441708],4.568510,[0.5069299169725915]
4,MultinomialNB2,[0.011855141944311738],[0.007543934847835405],[0.28796015325670493],[0.006108374384236453],0.323445,[0.006105895149498925]
5,Logistic regression2,[0.5242737247001812],[0.33964852121731676],[0.83377276155512],[0.3994088669950739],7.205719,[0.37586439901351976]


df scores jaccard_score